In [1]:
#Title: 
#What is the sex of an individual who is diagnosed with heart disease predicted by …. Symptoms?


#Intro:
#-describe heart disease
#-background info
#-gender difference


#Preliminary exploratory data analysis:


#Methods:


#Expected Outcomes and Predicitions 


# Using Symptoms to Predict the Sex of an Individual with Heart Disease
Dsci 100-006, Group 2

## Introduction

-intro here

In [ ]:
### Run this cell before continuing. 
library(tidyverse)
library(tidymodels)
library(repr)
source("cleanup.R")
options(repr.matrix.max.rows = 10)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ purrr     1.0.2
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [1]:
library(readr)

# heart_data <- read_csv("data goes here")
# head(heart_data)